# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ust-nera,64.57,143.20,32.68,92,100,0.98,RU,1602687722
1,1,villa bruzual,9.33,-69.12,85.77,70,25,2.98,VE,1602687724
2,2,ilulissat,69.22,-51.10,33.80,80,75,2.24,GL,1602687700
3,3,marzuq,14.40,46.47,81.41,18,0,5.93,YE,1602687729
4,4,lagoa,39.05,-27.98,68.63,73,0,7.61,PT,1602687731


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
humidity = cities_df["Humidity"].astype(float)


fig = gmaps.figure(zoom_level=1, center=(0, 0))

# Create a humidity Heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# ideal weather conditions
ideal_max_temp = 80
ideal_min_temp = 70
ideal_wind_speed = 5
ideal_cloud = 0

# create dataframe to satisfy ideal weather conditions
vacation_df = cities_df.loc[(cities_df['Max Temp'] >= ideal_min_temp) &
                           (cities_df['Max Temp'] < ideal_max_temp) &
                           (cities_df['Wind Speed'] <= ideal_wind_speed) &
                           (cities_df['Cloudiness'] <= ideal_cloud)]

#drop any rows will null values
vacation_df.dropna()
vacation_df

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
60,60,tura,25.52,90.22,74.98,94,0,2.24,IN,1602687871
266,266,alice springs,-23.70,133.88,75.00,32,0,4.70,AU,1602688353
418,418,pelotas,-31.77,-52.34,72.68,54,0,2.73,BR,1602688709
491,491,whittlesea,-32.18,26.82,74.80,18,0,2.51,ZA,1602688874


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# add column for Hotel Name
hotel_df = pd.DataFrame({"City": vacation_df["City"],
                         "Country": vacation_df["Country"],
                         "Lat": vacation_df["Lat"],
                         "Lng": vacation_df["Lng"]
                        })
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# set the base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"



In [7]:
# for each city in the ideal vacation spots
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    #     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))